<a href="https://colab.research.google.com/github/kubaBalga/Diplomka/blob/master/DiplomDraft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Necessary imports

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage import io
from skimage.color import label2rgb
from skimage.color.colorlabel import rgb2gray
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float

Get 256 shades of gray (in RGB format), randomly ordered

Intensity is scaled into interval [0, 1)

In [14]:
def get_random_grayscale():
  random_colors = []
  for i in range(256):
    brightness = ((i * 7229) % 256) / 256
    random_colors.append((brightness, brightness, brightness))
  return random_colors

Show calculated layer as grayscale image, save both image and numeric map

In [15]:
def show_and_save(layer_image, layer_map, layer_index):
    # plot
    fig_0 = plt.figure("Layer " + str(layer_index))
    fig_0.add_subplot().imshow(layer_image)
    plt.axis("off")

    # save image
    pil_image = Image.fromarray(obj=layer_image, mode="L")
    pil_image.save("segmentation_" + str(layer_index) + ".png")

    # save numeric map
    numeric_map = layer_map.tobytes()
    with open("segmentation_" + str(layer_index) + ".bin", "wb") as binary_file:
        binary_file.write(numeric_map)

Normalize numbering of segments to continuous - necessary for import

In [20]:
def normalize(map):
    normalization_dictionary = {}
    last_number = 0
    for i in range(map.shape[0]):
        for j in range(map.shape[1]):
            if map[i, j] not in normalization_dictionary:
                normalization_dictionary[map[i, j]] = last_number
                print(map[i, j], last_number)
                last_number += 1
            map[i, j] = normalization_dictionary[map[i, j]]
    print("------------")

Create hierarchical SLIC segmentation of input image
- Input: Path to 8-bit grayscale image, with resolution 768*512
- Effect: Creation of 7 segmentation maps (segmentation_0.bin - segmentation_6.bin), with corresponding map images


In [17]:
def calculate_subsegments(image, parental_map, seg_count, compactness, slic_zero):
    child_map = np.array([[0] * parental_map.shape[1] for i in range(parental_map.shape[0])], dtype=np.uint32)
    for segment_idx in range(0, seg_count):
      segment_mask = parental_map == segment_idx
      if (np.count_nonzero(segment_mask) == 0):
        continue
      # higher level of noise - increase compactness
      temp = slic(image, n_segments = 4, sigma = 5, compactness = compactness, mask = segment_mask, slic_zero=slic_zero).astype(np.uint32)
      for i in range(len(temp)):
        for j in range(len(temp[i])):
          # for each segmented subsegment, add offset for total numbering
          if temp[i][j] != 0:
            temp[i][j] += 4 * segment_idx - 1
      child_map += temp
    normalize(child_map)
    layer_image = np.round(rgb2gray(label2rgb(child_map, colors = get_random_grayscale())) * 256).astype(np.uint8)
    return child_map, layer_image

In [18]:
def make_segmentation(image_source):
    # load the image and convert it to a floating point data type
    image = img_as_float(io.imread(image_source))

    # calculate initial segments
    map_0 = slic(image, n_segments = 6, sigma = 5, compactness = 0.03, slic_zero=False).astype(np.uint32)
    layer_image = np.round(rgb2gray(label2rgb(map_0, colors = get_random_grayscale())) * 256).astype(np.uint8)
    show_and_save(layer_image, map_0, 0)

    # calculate 1-ary segments
    map_1, layer_image = calculate_subsegments(image, map_0, 6, 0.3, False)
    show_and_save(layer_image, map_1, 1)

    # calculate 2-ary segments
    map_2, layer_image = calculate_subsegments(image, map_1, 24, 1, True)
    show_and_save(layer_image, map_2, 2)

    # calculate 3-ary segments
    map_3, layer_image = calculate_subsegments(image, map_2, 96, 1, True)
    show_and_save(layer_image, map_3, 3)

    # calculate 4-ary segments
    map_4, layer_image = calculate_subsegments(image, map_3, 384, 1, True)
    show_and_save(layer_image, map_4, 4)

    # calculate 5-ary segments
    map_5, layer_image = calculate_subsegments(image, map_4, 1536, 1, True)
    show_and_save(layer_image, map_5, 5)

    # calculate 6-ary segments
    map_6, layer_image = calculate_subsegments(image, map_5, 6144, 1, True)
    show_and_save(layer_image, map_6, 6)

In [ ]:
make_segmentation("image.png")